# Bra detection

## Import libraries

In [ ]:
!pip install tensorflow

In [1]:
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img , img_to_array

import pandas as pd 
import numpy as np

import os

In [2]:
def extract_label(img_path, flag):
    filename, extension = os.path.splitext(os.path.basename(img_path))
    
    subject_id, etc = filename.split("__") # 2 underscore
    bra = 0
    
    if flag:
        breast_size, waist_size, band_size, cup_size, with_bra = etc.split('_')        

        if with_bra == 'true':
            bra = 1
        else:
            bra = 0

        return bra

In [3]:
bra_path = "./dataset/with_bra"
without_bra_path = "./dataset/without_bra"

images = []
labels = []

In [4]:
for image in os.listdir(without_bra_path):
    img = cv2.imread(os.path.join(without_bra_path, image), cv2.IMREAD_COLOR)
        
    # flag = 0 -> no bra
    # flag = 1 -> has bra
    label = extract_label(os.path.join(without_bra_path, image), False)
    
    labels.append(label)
    images.append(img);


In [5]:
for image in os.listdir(without_bra_path):
    img = cv2.imread(os.path.join(without_bra_path, image), cv2.IMREAD_COLOR)
        
    # flag = 0 -> no bra
    # flag = 1 -> has bra
    label = extract_label(os.path.join(without_bra_path, image), False)
    
    labels.append(label)
    images.append(img);

### Convert image arry to np array

In [7]:
np_images = np.array(images, dtype="object")
np_labels = np.array(labels)

## MobileNetV2 acrchitecture

In [ ]:
model = Sequential()
model.add(Conv2D(512,(2,2),padding="same",activation="relu",input_shape=(224,224,3)))
model.add(MaxPooling2D((2,2),strides=2))

model.add(Conv2D(256,(2,2),padding="same",activation="relu"))
model.add(MaxPooling2D((2,2),strides=2))

model.add(Conv2D(128,(2,2),padding="same",activation="relu"))
model.add(MaxPooling2D((2,2),strides=2))

model.add(Conv2D(64,(2,2),padding="same",activation="relu"))
model.add(MaxPooling2D((2,2),strides=2))

model.add(Conv2D(32,(2,2),padding="same",activation="relu"))
model.add(MaxPooling2D((2,2),strides=2))

#model.add(BatchNormalization())

model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(2,activation="softmax"))

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

In [ ]:
start = time.time()

history = model.fit(img_gen.flow(X_train,y_train,batch_size=5),
                    steps_per_epoch=300,
                    validation_data=(X_test,y_test),
                    validation_steps=300,
                    epochs=45)
end = time.time()
print("Total train time: ",(end-start)/60," mins")

In [ ]:
def plot_graph(history,string):
    plt.figure(figsize=(16,7))
    plt.plot(history.history[string],label=str(string))
    plt.plot(history.history["val_"+str(string)],label=str(string))
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string,"val_"+string])
    plt.show()

In [ ]:
plot_graph(history,"accuracy")
plot_graph(history,"loss")